<a id="title_ID"></a>
# JWST Pipeline Validation Notebook: NIRCam, calwebb_image3, source_catalog

<span style="color:red"> **Instruments Affected**</span>: e.g., FGS, MIRI, NIRCam, NIRISS, NIRSpec 

### Table of Contents

<div style="text-align: left"> 
    
<br> [Introduction](#intro)
<br> [JWST CalWG Algorithm](#algorithm)
<br> [Defining Terms](#terms)
<br> [Test Description](#description)
<br> [Data Description](#data_descr)
<br> [Set up Temporary Directory](#tempdir)
<br> [Imports](#imports)
<br> [Loading the Data](#data_load)
<br> [Run the Image3Pipeline](#pipeline)
<br> [Perform Visual Inspection](#visualization) 
<br> [Manually Find Matches](#manual)
<br> [About This Notebook](#about)
<br>    

</div>

<a id="intro"></a>
# Introduction

This is the NIRCam validation notebook for the Source Catalog step, which generates a catalog based on input exposures.

* Step description: https://jwst-pipeline.readthedocs.io/en/latest/jwst/source_catalog/index.html

* Pipeline code: https://github.com/spacetelescope/jwst/tree/master/jwst/source_catalog

[Top of Page](#title_ID)

<a id="algorithm"></a>
# JWST CalWG Algorithm

This is the NIRCam imaging validation notebook for the Source Catalog step, which uses image combinations or stacks of overlapping images to generate "browse-quality" source catalogs.  Having automated source catalogs will help accelerate the science output of JWST. The source catalogs should include both point and "slightly" extended sources at a minimum.  The catalog should provide an indication if the source is a point or an extended source. For point sources, the source catalog should include measurements corrected to infinite aperture using aperture corrections provided by a reference file.  

See: 
* https://outerspace.stsci.edu/display/JWSTCC/Vanilla+Point+Source+Catalog


[Top of Page](#title_ID)

<a id="terms"></a>
# Defining Terms

* JWST: James Webb Space Telescope

* NIRCam: Near-Infrared Camera


[Top of Page](#title_ID)

<a id="description"></a>
# Test Description

Here we generate the source catalog and visually inspect a plot of the image with the source catalog overlaid. We also look at some other diagnostic plots and then cross-check the output catalog against Mirage catalog inputs. 


[Top of Page](#title_ID)

<a id="data_descr"></a>
# Data Description

The set of data used in this test were created with the Mirage simulator. The simulator created a NIRCam imaging mode exposures for the short wave NRCA1 detector. 


[Top of Page](#title_ID)

<a id="tempdir"></a>
# Set up Temporary Directory
The following cell sets up a temporary directory (using python's `tempfile.TemporaryDirectory()`), and changes the script's active directory into that directory (using python's `os.chdir()`). This is so that, when the notebook is run through, it will download files to (and create output files in) the temporary directory rather than in the notebook's directory. This makes cleanup significantly easier (since all output files are deleted when the notebook is shut down), and also means that different notebooks in the same directory won't interfere with each other when run by the automated webpage generation process.

If you want the notebook to generate output in the notebook's directory, simply don't run this cell.

If you have a file (or files) that are kept in the notebook's directory, and that the notebook needs to use while running, you can copy that file into the directory (the code to do so is present below, but commented out).

In [1]:
#****
#
# Set this variable to False to not use the temporary directory
#
#****
use_tempdir = True

# Create a temporary directory to hold notebook output, and change the working directory to that directory.
from tempfile import TemporaryDirectory
import os
import shutil

if use_tempdir:
    data_dir = TemporaryDirectory()

    # If you have files that are in the notebook's directory, but that the notebook will need to use while
    # running, copy them into the temporary directory here.
    #
    # files = ['name_of_file']
    # for file_name in files:
    #     shutil.copy(file_name, os.path.join(data_dir.name, file_name))

    # Save original directory
    orig_dir = os.getcwd()

    # Move to new directory
    os.chdir(data_dir.name)

# For info, print out where the script is running
print("Running in {}".format(os.getcwd()))

Running in /data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpjuyrb665


[Top of Page](#title_ID)

## If Desired, set up CRDS to use a local cache

By default, the notebook template environment sets up its CRDS cache (the "CRDS_PATH" environment variable) in /grp/crds/cache. However, if the notebook is running on a local machine without a fast and reliable connection to central storage, it makes more sense to put the CRDS cache locally. Currently, the cell below offers several options, and will check the supplied boolean variables one at a time until one matches.

* if `use_local_crds_cache` is False, then the CRDS cache will be kept in /grp/crds/cache
* if `use_local_crds_cache` is True, the CRDS cache will be kept locally
  * if `crds_cache_tempdir` is True, the CRDS cache will be kept in the temporary directory
  * if `crds_cache_notebook_dir` is True, the CRDS cache will be kept in the same directory as the notebook.
  * if `crds_cache_home` is True, the CRDS cache will be kept in $HOME/crds/cache
  * if `crds_cache_custom_dir` is True, the CRDS cache will be kept in whatever is stored in the 
    `crds_cache_dir_name` variable.

If the above cell (creating a temporary directory) is not run, then setting `crds_cache_tempdir` to True will store the CRDS cache in the notebook's directory (the same as setting `crds_cache_notebook_dir` to True).

In [2]:
import os

# Choose CRDS cache location
use_local_crds_cache = False
crds_cache_tempdir = False
crds_cache_notebook_dir = False
crds_cache_home = False
crds_cache_custom_dir = False
crds_cache_dir_name = ""

if use_local_crds_cache:
    if crds_cache_tempdir:
        os.environ['CRDS_PATH'] = os.path.join(os.getcwd(), "crds")
    elif crds_cache_notebook_dir:
        try:
            os.environ['CRDS_PATH'] = os.path.join(orig_dir, "crds")
        except Exception as e:
            os.environ['CRDS_PATH'] = os.path.join(os.getcwd(), "crds")
    elif crds_cache_home:
        os.environ['CRDS_PATH'] = os.path.join(os.environ['HOME'], 'crds', 'cache')
    elif crds_cache_custom_dir:
        os.environ['CRDS_PATH'] = crds_cache_dir_name

[Top of Page](#title_ID)

<a id="imports"></a>
# Imports
List the package imports and why they are relevant to this notebook.


* astropy for various tools and packages
* inspect to get the docstring of our objects.
* IPython.display for printing markdown output
* jwst.datamodels for JWST Pipeline data models
* jwst.module.PipelineStep is the pipeline step being tested
* matplotlib.pyplot.plt to generate plot

In [3]:
# plotting, the inline must come before the matplotlib import
%matplotlib inline
# %matplotlib notebook

# These gymnastics are needed to make the sizes of the figures
# be the same in both the inline and notebook versions
%config InlineBackend.print_figure_kwargs = {'bbox_inches': None}
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80

from matplotlib import pyplot as plt
import matplotlib.patches as patches

params = {'legend.fontsize': 6,
          'figure.figsize': (8, 8),
          'figure.dpi': 150,
         'axes.labelsize': 6,
         'axes.titlesize': 6,
         'xtick.labelsize':6,
         'ytick.labelsize':6}
plt.rcParams.update(params)

# Box download imports 
from astropy.utils.data import download_file
from pathlib import Path
from shutil import move
from os.path import splitext

# python general
import os
import numpy as np

# astropy modules
import astropy
from astropy.io import fits
from astropy.table import QTable, Table, vstack, unique
from astropy.wcs.utils import skycoord_to_pixel
from astropy.coordinates import SkyCoord
from astropy.visualization import simple_norm
from astropy import units as u
import photutils

# jwst 
from jwst.pipeline import calwebb_image3
from jwst import datamodels

In [4]:
def create_image(data_2d, xpixel=None, ypixel=None, title=None):
    ''' Function to generate a 2D image of the data, 
    with an option to highlight a specific pixel.
    '''
    
    fig = plt.figure(figsize=(8, 8))
    ax = plt.subplot()

    norm = simple_norm(data_2d, 'sqrt', percent=99.)
    
    plt.imshow(data_2d, norm=norm, origin='lower', cmap='gray')
    
    if xpixel and ypixel:
        plt.plot(xpixel, ypixel, marker='o', color='red', label='Selected Pixel')

    plt.xlabel('Pixel column')
    plt.ylabel('Pixel row')
    
    if title:
        plt.title(title)

    plt.subplots_adjust(left=0.15)
    plt.colorbar(label='MJy/sr')

In [5]:
def create_image_with_cat(data_2d, catalog, flux_limit=None, title=None):
    ''' Function to generate a 2D image of the data, 
    with sources overlaid.
    '''
    
    fig = plt.figure(figsize=(8, 8))
    ax = plt.subplot()

    norm = simple_norm(data_2d, 'sqrt', percent=99.)
    
    plt.imshow(data_2d, norm=norm, origin='lower', cmap='gray')
    
    for row in catalog:
        if flux_limit:
            if np.isnan(row['aper_total_flux']):
                pass
            else:
                if row['aper_total_flux'] > flux_limit:
                    plt.plot(row['xcentroid'], row['ycentroid'], marker='o', markersize='3', color='red')
        else:
            plt.plot(row['xcentroid'], row['ycentroid'], marker='o', markersize='1', color='red')

    plt.xlabel('Pixel column')
    plt.ylabel('Pixel row')
    
    if title:
        plt.title(title)

    plt.subplots_adjust(left=0.15)
    plt.colorbar(label='MJy/sr')

In [6]:
def create_scatterplot(catalog_colx, catalog_coly, title=None):
    ''' Function to generate a generic scatterplot.
    '''
    
    fig = plt.figure(figsize=(8, 8))
    ax = plt.subplot()
    ax.scatter(catalog_colx,catalog_coly) 
    plt.xlabel(catalog_colx.name)
    plt.ylabel(catalog_coly.name)

    
    if title:
        plt.title(title)

In [7]:
def get_input_table(sourcelist):
    '''Function to read in and access the simulator source input files.'''

    all_source_table = Table()

    # point source and galaxy source tables have different headers
    # change column headers to match for filtering later
    if "point" in sourcelist:
        col_names = ["RA", "Dec", "RA_degrees", "Dec_degrees",
                     "PixelX", "PixelY", "Magnitude",
                     "counts_sec", "counts_frame"]
    elif "galaxy" in sourcelist:
        col_names = ["PixelX", "PixelY", "RA", "Dec",
                     "RA_degrees", "Dec_degrees", "V2", "V3", "radius",
                     "ellipticity", "pos_angle", "sersic_index",
                     "Magnitude", "countrate_e_s", "counts_per_frame_e"]
    else:
        print('Error! Source list column names need to be defined.')
        sys.exit(0)

    # read in the tables
    input_source_table = Table.read(sourcelist,format='ascii')
    orig_colnames = input_source_table.colnames

    # only grab values for source catalog analysis
    short_source_table = Table({'In_RA': input_source_table['RA_degrees'],
                              'In_Dec': input_source_table['Dec_degrees']},
                              names=['In_RA', 'In_Dec'])

    # combine source lists into one master list
    all_source_table = vstack([all_source_table, short_source_table])

    # set up columns to track which sources were detected by Photutils
    all_source_table['Out_RA'] = np.nan
    all_source_table['Out_Dec'] = np.nan
    all_source_table['Detected'] = 'N'
    all_source_table['RA_Diff'] = np.nan
    all_source_table['Dec_Diff'] = np.nan

    # filter by RA, Dec (for now)
    no_duplicates = unique(all_source_table,keys=['In_RA','In_Dec'])

    return no_duplicates

[Top of Page](#title_ID)

<a id="data_load"></a>
# Loading the Data

The simulated exposures used for this test are stored in Box. Grab them. 

In [8]:
def get_box_files(file_list):
    for box_url,file_name in file_list:
        if 'https' not in box_url:
            box_url = 'https://stsci.box.com/shared/static/' + box_url
        downloaded_file = download_file(box_url)
        if Path(file_name).suffix == '':
            ext = splitext(box_url)[1]
            file_name += ext
        move(downloaded_file, file_name)

file_urls = ['https://stsci.box.com/shared/static/72fds4rfn4ppxv2tuj9qy2vbiao110pc.fits', 
             'https://stsci.box.com/shared/static/gxwtxoz5abnsx7wriqligyzxacjoz9h3.fits', 
             'https://stsci.box.com/shared/static/tninaa6a28tsa1z128u3ffzlzxr9p270.fits',
             'https://stsci.box.com/shared/static/g4zlkv9qi0vc5brpw2lamekf4ekwcfdn.json',
             'https://stsci.box.com/shared/static/kvusxulegx0xfb0uhdecu5dp8jkeluhm.list']

file_names = ['jw00042002001_01101_00004_nrca5_cal.fits', 
             'jw00042002001_01101_00005_nrca5_cal.fits', 
             'jw00042002001_01101_00006_nrca5_cal.fits',
             'level3_lw_imaging_files_asn.json',
             'jw00042002001_01101_00004_nrca5_uncal_galaxySources.list']

box_download_list = [(url,name) for url,name in zip(file_urls,file_names)]

In [9]:
get_box_files(box_download_list)

[Top of Page](#title_ID)

<a id="pipeline"></a>
# Run the Image3Pipeline

Run calwebb_image3 to get the output source catalog and the final 2D image. 

In [10]:
img3 = calwebb_image3.Image3Pipeline()

2021-10-02 12:04:31,549 - stpipe.Image3Pipeline - INFO - Image3Pipeline instance created.


2021-10-02 12:04:31,551 - stpipe.Image3Pipeline.assign_mtwcs - INFO - AssignMTWcsStep instance created.


2021-10-02 12:04:31,553 - stpipe.Image3Pipeline.tweakreg - INFO - TweakRegStep instance created.


2021-10-02 12:04:31,555 - stpipe.Image3Pipeline.skymatch - INFO - SkyMatchStep instance created.


2021-10-02 12:04:31,557 - stpipe.Image3Pipeline.outlier_detection - INFO - OutlierDetectionStep instance created.


2021-10-02 12:04:31,558 - stpipe.Image3Pipeline.resample - INFO - ResampleStep instance created.


2021-10-02 12:04:31,560 - stpipe.Image3Pipeline.source_catalog - INFO - SourceCatalogStep instance created.


In [11]:
img3.assign_mtwcs.skip=True
img3.save_results=True
img3.resample.save_results=True
img3.source_catalog.snr_threshold = 5
img3.source_catalog.save_results=True

In [12]:
img3.run(file_names[3])

2021-10-02 12:04:31,682 - stpipe.Image3Pipeline - INFO - Step Image3Pipeline running with args ('level3_lw_imaging_files_asn.json',).


2021-10-02 12:04:31,689 - stpipe.Image3Pipeline - INFO - Step Image3Pipeline parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'steps': {'assign_mtwcs': {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': True, 'output_use_index': True, 'save_results': False, 'skip': True, 'suffix': 'assign_mtwcs', 'search_output_file': True, 'input_dir': ''}, 'tweakreg': {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': True, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'save_catalogs': False, 'catalog_format': 'ecsv', 'kernel_fwhm': 2.5, 'snr_threshold': 10.0, 'brightest': 

2021-10-02 12:04:32,217 - stpipe.Image3Pipeline - INFO - Prefetching reference files for dataset: 'jw00042002001_01101_00004_nrca5_cal.fits' reftypes = ['abvegaoffset', 'apcorr', 'drizpars']


2021-10-02 12:04:32,691 - stpipe.Image3Pipeline - INFO - Prefetch for ABVEGAOFFSET reference file is '/grp/crds/cache/references/jwst/jwst_nircam_abvegaoffset_0001.asdf'.


2021-10-02 12:04:32,695 - stpipe.Image3Pipeline - INFO - Prefetch for APCORR reference file is '/grp/crds/cache/references/jwst/jwst_nircam_apcorr_0004.fits'.


...

2021-10-02 12:06:48,041 - stpipe.Image3Pipeline.source_catalog - INFO - Subarray: FULL


2021-10-02 12:06:48,086 - stpipe.Image3Pipeline.source_catalog - INFO - AB to Vega magnitude offset 2.82390


2021-10-02 12:06:49,479 - stpipe.Image3Pipeline.source_catalog - WARNING - /data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks/lib/python3.9/site-packages/jwst/source_catalog/source_catalog.py:422: AstropyDeprecationWarning: "filter_kernel" was deprecated in version 1.2 and will be removed in a future version. Use argument "kernel" instead.
  segm = detect_sources(data, threshold, npixels, filter_kernel=kernel,



2021-10-02 12:06:49,832 - stpipe.Image3Pipeline.source_catalog - INFO - Detected 2370 sources


ValueError: too many values to unpack (expected 2)

[Top of Page](#title_ID)

<a id="visualization"></a>
# Perform Visual Inspection

Perform the visual inspection of the catalog and the final image. 

In [ ]:
catalog = Table.read("lw_imaging_cat.ecsv")

In [ ]:
combined_image = datamodels.ImageModel("lw_imaging_i2d.fits")

In [ ]:
create_image(combined_image.data, title="Final combined NIRCam image")

In [ ]:
create_image_with_cat(combined_image.data, catalog, title="Final image w/ catalog overlaid")

In [ ]:
catalog

In [ ]:
create_scatterplot(catalog['label'], catalog['aper_total_flux'],title='Total Flux in '+str(catalog['aper_total_flux'].unit))

In [ ]:
create_scatterplot(catalog['label'], catalog['aper_total_abmag'],title='Total AB mag')

[Top of Page](#title_ID)

<a id="manual"></a>
# Manually Find Matches 

Since this is a simulated data set, we can compare the output catalog information from the pipeline with the input catalog information used to create the simulation. Grab the input catalog RA, Dec values and the output catalog RA, Dec values.

In [ ]:
test_outputs = get_input_table(file_names[4])
in_ra = test_outputs['In_RA'].data
in_dec = test_outputs['In_Dec'].data
out_ra = catalog['sky_centroid'].ra.deg
out_dec = catalog['sky_centroid'].dec.deg

Set the tolerance and initialize our counters. 

In [ ]:
tol = 1.e-3
found_count=0
multiples_count=0
missed_count=0

Below we loop through the input RA, Dec values and compare them to the RA, Dec values in the output catalog. For cases where there are multiple matches for our tolerance level, count those cases. 

In [ ]:
for ra,dec,idx in zip(in_ra, in_dec,range(len(test_outputs))):

    match = np.where((np.abs(ra-out_ra) < tol) & (np.abs(dec-out_dec) < tol))
    
    if np.size(match) == 1: 
        found_count +=1 
        test_outputs['Detected'][idx] = 'Y'
        test_outputs['Out_RA'][idx] = out_ra[match]
        test_outputs['Out_Dec'][idx] = out_dec[match]
        test_outputs['RA_Diff'][idx] = np.abs(ra-out_ra[match])
        test_outputs['Dec_Diff'][idx] = np.abs(dec-out_dec[match])  

    if np.size(match) > 1:  
        multiples_count +=1       
        
    if np.size(match) < 1:
        missed_count +=1

Let's see how it did. 

In [ ]:
total_percent_found = (found_count/len(test_outputs))*100

print('\n')
print('SNR threshold used for pipeline: ',img3.source_catalog.snr_threshold)
print('Total found:',found_count)
print('Total missed:',missed_count)
print('Number of multiples: ',multiples_count)
print('Total number of input sources:',len(test_outputs))
print('Total number in output catalog:',len(catalog))
print('Total percent found:',total_percent_found)
print('\n')

### Use photutils to find catalog matches

Photutils includes a package to match sources between catalogs by providing a max separation value. Set that value and compare the two catalogs.

In [ ]:
catalog_in = SkyCoord(ra=in_ra*u.degree, dec=in_dec*u.degree)
catalog_out = SkyCoord(ra=out_ra*u.degree, dec=out_dec*u.degree)

In [ ]:
max_sep = 1.0 * u.arcsec

In [ ]:
# idx, d2d, d3d = cat_in.match_to_catalog_3d(cat_out)
idx, d2d, d3d = catalog_in.match_to_catalog_sky(catalog_out)
sep_constraint = d2d < max_sep
catalog_in_matches = catalog_in[sep_constraint]
catalog_out_matches = catalog_out[idx[sep_constraint]]

Now, ```catalog_in_matches``` and ```catalog_out_matches``` are the matched sources in ```catalog_in``` and ```catalog_out```, respectively, which are separated less than our ```max_sep``` value.

In [ ]:
print('Number of matched sources using max separation of '+str(max_sep)+': ',len(catalog_out_matches))

<a id="about_ID"></a>
## About this Notebook
**Author:** Alicia Canipe, Senior Staff Scientist, NIRCam
<br>**Updated On:** 05/26/2021

[Top of Page](#title_ID)
<img style="float: right;" src="./stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="stsci_pri_combo_mark_horizonal_white_bkgd" width="200px"/> 